# Get Azure ML Workspace

In [1]:
from azureml.core import Workspace, Environment, Experiment
ws = Workspace.from_config()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Resource Group: ' + ws.resource_group, 
      sep = '\n')

Workspace Name: iiotml
Azure Region: westus2
Resource Group: iiotsample


# Create Conda Environment with Dependencies

In [2]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment(name="train-env")
train_conda_deps = CondaDependencies.create(pip_packages=['pandas','azureml-core','azureml-dataset-runtime[fuse]','numpy','scikit-learn','azure-kusto-data[pandas]','lightgbm'])
train_env.python.conda_dependencies = train_conda_deps

# Create Retraining Pipeline

In [3]:
from azureml.core.compute import AmlCompute
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

compute = AmlCompute(ws, "cpu-cluster")

retrainRunConfig = RunConfiguration()
retrainRunConfig.target = compute
retrainRunConfig.environment = train_env

modelname_param = PipelineParameter(name="modelname", default_value='iiot-quality-lgbm')
selected_features_param = PipelineParameter(name="selectedfeatures", default_value="S16,S20,S19,S18,S29,S41,S9,S10,S8,S11,S14,S13,S28,S15,S26,S33,S7,S3,S39")

buildDatasetStep = PythonScriptStep(name="build-datasets",
    script_name="build-datasets.py", 
    runconfig = retrainRunConfig, 
    compute_target=compute, 
    source_directory=".",
    allow_reuse=False)

buildModelStep = PythonScriptStep(name="build-model",
    arguments=["--modelname", modelname_param, "--selectedfeatures", selected_features_param],
    script_name="build-model.py", 
    runconfig = retrainRunConfig, 
    compute_target=compute, 
    source_directory=".",
    allow_reuse=False)

buildModelStep.run_after(buildDatasetStep)

In [4]:
retrainingPipeline = Pipeline(workspace=ws, steps=[buildModelStep])

retrainingPipeline.validate()

Step build-model is ready to be created [30249118]
Step build-datasets is ready to be created [5a7ac826]


[]

In [ ]:
retrainPipelineRun = Experiment(ws, 'Model-Training-Pipeline').submit(retrainingPipeline)

# Publish Retraining Pipeline

In [7]:
publishedPipeline = retrainPipelineRun.publish_pipeline(name="iiot-quality-training", description="sample pipeline to retrain iiot quality prediction model",version="0.1")
print("Newly published pipeline id => ",publishedPipeline.id)
print("Newly published pipeline endpoint => ", publishedPipeline.endpoint)